[![](https://mermaid.ink/img/pako:eNptkc1Kw0AYRV9lmHX6AlkIbdPWjStdOcliaKZNoPkhThaSFEqlJdhNQF2IEolQUYTiX6WCxZdxkvEtTJNaIzirOd89DMN3Pdi2VAJF2HWwrYE9STZBdqroc_mRnt7ywYi_L9hoqoBKZQvUULpYprM4Dceg2iUmVQq9lqd1jwXjJAr5_SR5HH7FEYsm_UKorwSfD2N29JZcTfndtQ8kVGb2ELDZq1LWk-MBj0-KN33QQGX-R-dPMZuc_-hNVOY_popT_toWSsxcWBik8zCJLtdRo4gKaJaRlcM2YhcRGz6n85vNghQoQIM4BtbVbI_eSpch1YhBZChmV5V0sNujMpTJfqZil1q7h2YbitRxiQBdW8WUSDpOGjCg2MG9g820oerUcjZDG5v7lvUrkTzeKQrMexSgY7ldbW30v0kGPk8?type=png)](https://mermaid.live/edit#pako:eNptkc1Kw0AYRV9lmHX6AlkIbdPWjStdOcliaKZNoPkhThaSFEqlJdhNQF2IEolQUYTiX6WCxZdxkvEtTJNaIzirOd89DMN3Pdi2VAJF2HWwrYE9STZBdqroc_mRnt7ywYi_L9hoqoBKZQvUULpYprM4Dceg2iUmVQq9lqd1jwXjJAr5_SR5HH7FEYsm_UKorwSfD2N29JZcTfndtQ8kVGb2ELDZq1LWk-MBj0-KN33QQGX-R-dPMZuc_-hNVOY_popT_toWSsxcWBik8zCJLtdRo4gKaJaRlcM2YhcRGz6n85vNghQoQIM4BtbVbI_eSpch1YhBZChmV5V0sNujMpTJfqZil1q7h2YbitRxiQBdW8WUSDpOGjCg2MG9g820oerUcjZDG5v7lvUrkTzeKQrMexSgY7ldbW30v0kGPk8)

In [ ]:
%%capture
!pip install google-generativeai
!pip install yfinance pandas_ta

In [ ]:
import yfinance as yf
import pandas_ta as ta
import pandas as pd
import os
import google.generativeai as genai
from IPython.display import Markdown

In [ ]:
from google.colab import userdata
os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

In [ ]:
# 股票代码格式化函数
def format_stock_symbol(symbol):
    # 自动为纯数字的6位A股代码添加.SS后缀
    if symbol.isdigit() and len(symbol) == 6:
        return f"{symbol}.SS"
    return symbol

In [ ]:
# 股价走势查询函数
def get_stock_price_history(symbol, period="1mo"):
    try:
        stock = yf.Ticker(symbol)
        history = stock.history(period=period)
        return history.tail().to_dict('records')
    except Exception as e:
        return f"无法获取 {symbol} 的历史价格: {str(e)}"

In [ ]:
# 主交互系统
def main():
    print("欢迎使用股票市场助手（Gemini版）！")
    print("可查询功能：实时报价/财务数据/技术分析/综合评估")

    while True:
        symbol = input("\n请输入股票代码 (例如：600888 或 2330.TW) 或输入exit退出: ")
        
        if symbol.lower() in ['exit', 'quit']:
            break
            
        # 格式化股票代码
        formatted_symbol = format_stock_symbol(symbol)
        
        # 获取基础数据
        price_data = get_stock_price_history(formatted_symbol)
        info_data = get_stock_info(formatted_symbol)
        financial_data = get_financial_data(formatted_symbol)
        technical_data = get_technical_analysis(formatted_symbol)

        # 构建分析上下文
        context = {
            '基本信息': info_data,
            '财务数据': financial_data,
            '技术指标': technical_data.get('indicators', {}),
            '历史数据': technical_data.get('historical_data', [])
        }

        # 获取用户问题
        query = input("请输入您的分析问题 (例如：技术面分析/财务健康度/综合评估): ")

        # 使用Gemini进行分析
        analysis = analyze_with_gemini(query, context)
        display(Markdown(f"**Gemini分析结果:**\n{analysis}"))

if __name__ == "__main__":
    main()